# Redis Chat Message Store for Multi-Turn Conversations

This notebook demonstrates how to use **Redis as a message store** for managing conversation threads with AI agents using the Microsoft Agent Framework.

## Overview

Redis is an excellent choice for storing conversation history in production applications. It provides:

- **High-performance** in-memory data storage with persistence
- **Distributed** conversation management across multiple application instances
- **Automatic expiration** of old conversations with TTL (Time-To-Live)
- **Scalable** architecture for handling many concurrent conversations
- **Session management** across different user sessions and devices

## Key Features

### RedisChatMessageStore

The Agent Framework provides a built-in `RedisChatMessageStore` that:
- Stores messages in Redis with automatic serialization
- Supports thread-based conversation isolation
- Implements message limits for memory management
- Provides async operations for high performance
- Handles connection management and cleanup

### Use Cases

- **User session management**: Track conversations per user and session
- **Multi-device sync**: Access the same conversation from different devices
- **Load-balanced applications**: Share conversation state across app instances
- **Conversation persistence**: Restore conversations after application restarts
- **Analytics and monitoring**: Track conversation patterns and metrics

## 📖 Documentation

For more details, see the official documentation:
- [Multi-Turn Conversations](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)
- [Custom Message Stores](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python#custom-message-stores)

---

## Prerequisites

Before running this notebook, ensure you have:

1. **Redis server running** on localhost:6379 (or update the connection URL)
   ```bash
   # Using Docker:
   docker run -d -p 6379:6379 redis:latest
   
   # Or install Redis locally
   ```

2. **Environment variables configured** in `agent-framework/.env`:
   - `OPENAI_API_KEY` (or other chat client credentials)
   - `REDIS_URL` (optional, defaults to redis://localhost:6379)

3. **Required packages installed**:
   ```bash
   pip install agent-framework redis
   ```

## Setup and Imports

In [ ]:
# Import required libraries
import asyncio
import os
from uuid import uuid4
from pathlib import Path
from dotenv import load_dotenv

from agent_framework import AgentThread
from agent_framework.openai import OpenAIChatClient
from agent_framework.redis import RedisChatMessageStore
from openai import AsyncAzureOpenAI

# Load environment variables
env_path = Path(__file__).parent.parent / ".env" if "__file__" in globals() else Path("../.env")
load_dotenv(env_path)

# Create Azure OpenAI client to be reused across examples
azure_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY") or os.getenv("OPENAI_API_KEY"),
    api_version="2024-10-21",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

print("Environment loaded and Azure OpenAI client created")

## Example 1: Basic Redis Chat Message Store

This example demonstrates the basic usage of Redis for storing conversation messages.

### Key Concepts:
- **Auto-generated thread IDs**: Each conversation gets a unique identifier
- **Message persistence**: Messages are stored in Redis across interactions
- **Connection management**: Proper initialization and cleanup

In [ ]:
async def example_basic_redis_store():
    """Basic example of using Redis chat message store."""
    print("=== Basic Redis Chat Message Store Example ===")
    print()

    # Create Redis store with auto-generated thread ID
    redis_store = RedisChatMessageStore(
        redis_url="redis://localhost:6379",
        # thread_id will be auto-generated if not provided
    )

    print(f"✓ Created store with thread ID: {redis_store.thread_id}")
    print()

    # Create thread with Redis store
    thread = AgentThread(message_store=redis_store)

    # Create agent with Azure OpenAI
    agent = OpenAIChatClient(
        model_id=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o"),
        async_client=azure_client
    ).create_agent(
        name="RedisBot",
        instructions="You are a helpful assistant that remembers our conversation using Redis.",
    )

    # Have a conversation
    print("--- Starting conversation ---")
    print()
    
    query1 = "Hello! My name is Alice and I love pizza."
    print(f"User: {query1}")
    response1 = await agent.run(query1, thread=thread)
    print(f"Agent: {response1.text}")
    print()

    query2 = "What do you remember about me?"
    print(f"User: {query2}")
    response2 = await agent.run(query2, thread=thread)
    print(f"Agent: {response2.text}")
    print()

    # Show messages are stored in Redis
    messages = await redis_store.list_messages()
    print(f"📊 Total messages in Redis: {len(messages)}")
    print()

    # Cleanup
    await redis_store.clear()
    await redis_store.aclose()
    print("✓ Cleaned up Redis data")
    print()

In [ ]:
# Run the basic example
await example_basic_redis_store()

## Example 2: User Session Management

This example demonstrates how to manage user sessions with Redis, which is essential for multi-user applications.

### Key Concepts:
- **User-specific thread IDs**: Organize conversations by user and session
- **Message limits**: Control memory usage with `max_messages` parameter
- **Factory pattern**: Create stores dynamically for each user session

In [ ]:
async def example_user_session_management():
    """Example of managing user sessions with Redis."""
    print("=== User Session Management Example ===")
    print()

    user_id = "alice_123"
    session_id = f"session_{uuid4()}"

    # Create Redis store for specific user session
    def create_user_session_store():
        return RedisChatMessageStore(
            redis_url="redis://localhost:6379",
            thread_id=f"user_{user_id}_{session_id}",
            max_messages=10,  # Keep only last 10 messages
        )

    # Create agent with factory pattern
    # Create agent with factory pattern and Azure OpenAI\n    agent = OpenAIChatClient(\n        model_id=os.getenv(\"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME\", \"gpt-4o\"),\n        async_client=azure_client\n    ).create_agent(\n        name=\"SessionBot\",\n        instructions=\"You are a helpful assistant. Keep track of user preferences.\",\n        chat_message_store_factory=create_user_session_store,\n    )

    # Start conversation
    thread = agent.get_new_thread()

    print(f"✓ Started session for user {user_id}")
    if hasattr(thread.message_store, "thread_id"):
        print(f"✓ Thread ID: {thread.message_store.thread_id}")  # type: ignore[union-attr]
    print()

    # Simulate conversation
    queries = [
        "Hi, I'm Alice and I prefer vegetarian food.",
        "What restaurants would you recommend?",
        "I also love Italian cuisine.",
        "Can you remember my food preferences?",
    ]

    for i, query in enumerate(queries, 1):
        print(f"--- Message {i} ---")
        print(f"User: {query}")
        response = await agent.run(query, thread=thread)
        print(f"Agent: {response.text}")
        print()

    # Show persistent storage
    if thread.message_store:
        messages = await thread.message_store.list_messages()  # type: ignore[union-attr]
        print(f"📊 Messages stored for user {user_id}: {len(messages)}")
    print()

    # Cleanup
    if thread.message_store:
        await thread.message_store.clear()  # type: ignore[union-attr]
        await thread.message_store.aclose()  # type: ignore[union-attr]
    print("✓ Cleaned up session data")
    print()

In [ ]:
# Run the user session management example
await example_user_session_management()

## Example 3: Conversation Persistence Across Application Restarts

This example demonstrates one of Redis's most powerful features: maintaining conversation context even after application restarts.

### Key Concepts:
- **Persistent storage**: Conversations survive application lifecycle
- **Same thread ID**: Reconnecting to existing conversations
- **Stateless applications**: No need to keep conversations in memory

In [ ]:
async def example_conversation_persistence():
    """Example of conversation persistence across application restarts."""
    print("=== Conversation Persistence Example ===")
    print()

    conversation_id = "persistent_chat_001"

    # Phase 1: Start conversation
    print("--- Phase 1: Starting conversation ---")
    print()
    
    store1 = RedisChatMessageStore(
        redis_url="redis://localhost:6379",
        thread_id=conversation_id,
    )

    thread1 = AgentThread(message_store=store1)\n    agent = OpenAIChatClient(\n        model_id=os.getenv(\"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME\", \"gpt-4o\"),\n        async_client=azure_client\n    ).create_agent(\n        name=\"PersistentBot\",\n        instructions=\"You are a helpful assistant. Remember our conversation history.\",\n    )

    # Start conversation
    query1 = "Hello! I'm working on a Python project about machine learning."
    print(f"User: {query1}")
    response1 = await agent.run(query1, thread=thread1)
    print(f"Agent: {response1.text}")
    print()

    query2 = "I'm specifically interested in neural networks."
    print(f"User: {query2}")
    response2 = await agent.run(query2, thread=thread1)
    print(f"Agent: {response2.text}")
    print()

    message_count = len(await store1.list_messages())
    print(f"📊 Stored {message_count} messages in Redis")
    await store1.aclose()
    print("✓ Closed connection (simulating app shutdown)")
    print()

    # Phase 2: Resume conversation (simulating app restart)
    print("--- Phase 2: Resuming conversation (after 'restart') ---")
    print()
    
    store2 = RedisChatMessageStore(
        redis_url="redis://localhost:6379",
        thread_id=conversation_id,  # Same thread ID
    )

    thread2 = AgentThread(message_store=store2)

    # Continue conversation - agent should remember context
    query3 = "What was I working on before?"
    print(f"User: {query3}")
    response3 = await agent.run(query3, thread=thread2)
    print(f"Agent: {response3.text}")
    print()

    query4 = "Can you suggest some Python libraries for neural networks?"
    print(f"User: {query4}")
    response4 = await agent.run(query4, thread=thread2)
    print(f"Agent: {response4.text}")
    print()

    final_count = len(await store2.list_messages())
    print(f"📊 Total messages after resuming: {final_count}")
    print()

    # Cleanup
    await store2.clear()
    await store2.aclose()
    print("✓ Cleaned up persistent data")
    print()

In [ ]:
# Run the conversation persistence example
await example_conversation_persistence()

## Example 4: Thread Serialization with Redis

This example shows how to serialize and deserialize threads that use Redis storage.

### Key Concepts:
- **Lightweight serialization**: Only thread metadata is serialized, not messages
- **Message retrieval**: Messages are loaded from Redis when deserializing
- **State transfer**: Move conversations between different parts of your application

In [ ]:
async def example_thread_serialization():
    """Example of thread state serialization and deserialization."""
    print("=== Thread Serialization Example ===")
    print()

    # Create initial thread with Redis store
    original_store = RedisChatMessageStore(
        redis_url="redis://localhost:6379",
        thread_id="serialization_test",
        max_messages=50,
    )

    original_thread = AgentThread(message_store=original_store)\n\n    agent = OpenAIChatClient(\n        model_id=os.getenv(\"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME\", \"gpt-4o\"),\n        async_client=azure_client\n    ).create_agent(\n        name=\"SerializationBot\",\n        instructions=\"You are a helpful assistant.\",\n    )

    # Have initial conversation
    print("--- Initial conversation ---")
    query1 = "Hello! I'm testing serialization."
    print(f"User: {query1}")
    response1 = await agent.run(query1, thread=original_thread)
    print(f"Agent: {response1.text}")
    print()

    # Serialize thread state
    serialized_thread = await original_thread.serialize()
    print(f"📦 Serialized thread state: {serialized_thread}")
    print()
    print("💡 Note: With Redis, only thread metadata is serialized,")
    print("   not the actual messages (which remain in Redis)")
    print()

    # Close original connection
    await original_store.aclose()
    print("✓ Closed original connection")
    print()

    # Deserialize thread state (simulating loading from database/file)
    print("--- Deserializing thread state ---")
    print()

    # Create a new thread with the same Redis store type
    # This ensures the correct store type is used for deserialization
    restored_store = RedisChatMessageStore(redis_url="redis://localhost:6379")
    restored_thread = await AgentThread.deserialize(serialized_thread, message_store=restored_store)

    print("✓ Thread restored from serialized state")
    print()

    # Continue conversation with restored thread
    query2 = "Do you remember what I said about testing?"
    print(f"User: {query2}")
    response2 = await agent.run(query2, thread=restored_thread)
    print(f"Agent: {response2.text}")
    print()

    # Cleanup
    if restored_thread.message_store:
        await restored_thread.message_store.clear()  # type: ignore[union-attr]
        await restored_thread.message_store.aclose()  # type: ignore[union-attr]
    print("✓ Cleaned up serialization test data")
    print()

In [ ]:
# Run the thread serialization example
await example_thread_serialization()

## Example 5: Message Limits and Automatic Trimming

This example demonstrates how to control memory usage with message limits.

### Key Concepts:
- **max_messages parameter**: Limit the number of stored messages
- **Automatic trimming**: Old messages are removed when limit is reached
- **Memory management**: Prevent unbounded growth of conversation history

In [ ]:
async def example_message_limits():
    """Example of automatic message trimming with limits."""
    print("=== Message Limits Example ===")
    print()

    # Create store with small message limit
    store = RedisChatMessageStore(
        redis_url="redis://localhost:6379",
        thread_id="limits_test",
        max_messages=3,  # Keep only 3 most recent messages
    )

    thread = AgentThread(message_store=store)
    agent = OpenAIChatClient().create_agent(
        name="LimitBot",
        instructions="You are a helpful assistant with limited memory.",
    )

    # Send multiple messages to test trimming
    messages = [
        "Message 1: Hello!",
        "Message 2: How are you?",
        "Message 3: What's the weather?",
        "Message 4: Tell me a joke.",
        "Message 5: This should trigger trimming.",
    ]

    for i, query in enumerate(messages, 1):
        print(f"--- Sending message {i} ---")
        print(f"User: {query}")
        response = await agent.run(query, thread=thread)
        print(f"Agent: {response.text}")

        stored_messages = await store.list_messages()
        print(f"📊 Messages in store: {len(stored_messages)}")
        if len(stored_messages) > 0:
            print(f"   Oldest message: {stored_messages[0].text[:30]}...")
        print()

    # Final check
    final_messages = await store.list_messages()
    print(f"📊 Final message count: {len(final_messages)}")
    print(f"   (should be <= 6: 3 messages × 2 per exchange)")
    print()

    # Cleanup
    await store.clear()
    await store.aclose()
    print("✓ Cleaned up limits test data")
    print()

In [ ]:
# Run the message limits example
await example_message_limits()

## Running All Examples

Execute all examples in sequence to see the complete Redis integration:

In [ ]:
async def run_all_examples():
    """Run all Redis chat message store examples."""
    print("Redis Chat Message Store Examples")
    print("=" * 50)
    print()
    print("Prerequisites:")
    print("- Redis server running on localhost:6379")
    print("- OPENAI_API_KEY environment variable set")
    print("=" * 50)
    print()

    # Check prerequisites
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ ERROR: OPENAI_API_KEY environment variable not set")
        return

    try:
        # Test Redis connection
        test_store = RedisChatMessageStore(redis_url="redis://localhost:6379")
        connection_ok = await test_store.ping()
        await test_store.aclose()
        if not connection_ok:
            raise Exception("Redis ping failed")
        print("✓ Redis connection successful")
        print()
    except Exception as e:
        print(f"❌ ERROR: Cannot connect to Redis: {e}")
        print("Please ensure Redis is running on localhost:6379")
        print()
        print("To start Redis with Docker:")
        print("  docker run -d -p 6379:6379 redis:latest")
        return

    try:
        # Run all examples
        await example_basic_redis_store()
        await example_user_session_management()
        await example_conversation_persistence()
        await example_thread_serialization()
        await example_message_limits()

        print("=" * 50)
        print("✅ All examples completed successfully!")
        print("=" * 50)

    except Exception as e:
        print(f"❌ Error running examples: {e}")
        raise

In [ ]:
# Run all examples
await run_all_examples()

## Key Takeaways

### Advantages of Redis for Conversation Storage

1. **Performance**: In-memory storage provides sub-millisecond latency
2. **Persistence**: Optional disk persistence survives restarts
3. **Scalability**: Distributed across multiple app instances
4. **Session Management**: Thread IDs enable multi-user support
5. **Memory Control**: Message limits prevent unbounded growth

### Production Considerations

1. **Redis Configuration**:
   - Enable persistence (RDB or AOF)
   - Set appropriate memory limits
   - Configure TTL for automatic cleanup
   - Use Redis Cluster for high availability

2. **Connection Management**:
   - Use connection pooling
   - Handle reconnection on failures
   - Implement circuit breakers
   - Monitor connection health

3. **Security**:
   - Enable authentication (requirepass)
   - Use TLS for encrypted connections
   - Implement network isolation
   - Regular security updates

4. **Monitoring**:
   - Track memory usage
   - Monitor connection counts
   - Log slow queries
   - Set up alerts for failures

### Best Practices

1. **Thread ID Naming**: Use structured IDs (e.g., `user_{user_id}_session_{session_id}`)
2. **Message Limits**: Set appropriate `max_messages` to control memory
3. **Cleanup**: Always call `aclose()` to release connections
4. **Error Handling**: Wrap Redis operations in try-except blocks
5. **Testing**: Use a separate Redis database for testing

## Next Steps

- Learn about **[Custom Message Stores](2-custom_chat_message_store_thread.ipynb)** for other backends
- Explore **[Thread Suspend/Resume](4-suspend_resume_thread.ipynb)** patterns
- Review **[Multi-Turn Conversation Documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)**
- Check out **[Azure AI Thread Serialization](1-azure-ai-thread-serialization.ipynb)** for cloud-based storage